# Working with parameters

## Goal

- Read in parameters for pure substances and mixtures from json files, and
- create parameters for pure substances and mixtures within Python.
- For both regular parameters as well as via the homo-segmented group contribution method.
- Learn how to access information stored in parameter objects.

## Parameters and the equation of state

Before we can start to compute physical properties, two steps are needed:

1. Build a parameter object.
2. Instatiate the equation of state using the parameter object.

In principle, every implementation of an equation of state can manage parameters differently but typically the workflow is similar for each implementation.

For `pcsaft` we first generate the parameters object, `PcSaftParameters`, which we then use to generate the equation of state object using `EquationOfState.pcsaft()`.
The `PcSaftParameters` object is part of the `feos.pcsaft` module while `EquationOfState` is part of the `feos.eos` module.

In [24]:
from feos.pcsaft import PcSaftParameters, IdentifierOption, AssociationRecord

---
## Read parameters from json file(s)

The easiest way to create the `PcSaftParameters` object is to read information from one or more json files.

- To read information from a single file, use `PcSaftParameters.from_json`
- To read information from multiple files, use `PcSaftParameters.from_multiple_json`

### From a single file

#### Pure substance

Querying a substance from a file requires an *identifier*.
This identifier can be one of `Name`, `Cas`, `Inchi`, `IupacName`, `Formula`, or `Smiles` with `Name` (common english name) being the default.
We can change the identifier type usig the `search_option` argument with an `IdentifierOption` object. Given a list of identifiers and a path to the parameter file, we can conveniently generate our object.

In [3]:
# path to parameter file for substances that are non-associating, i.e. defined by three parameters: m, sigma, and epsilon_k.
file_na = '../parameters/pcsaft/gross2001.json' 

# a system containing a single substance, "methane", using "Name" as identifier (default)
parameters = PcSaftParameters.from_json(['methane'], pure_path=file_na)
parameters

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|methane|16.043|1|3.7039|150.03|0|0|0|0|1|1|

In [8]:
# a system containing a single substance, "methane", using "Smiles" ("C") as identifier
parameters = PcSaftParameters.from_json(['C'], pure_path=file_na, search_option=IdentifierOption.Smiles)
parameters

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|methane|16.043|1|3.7039|150.03|0|0|0|0|1|1|

#### Mixtures

Reading parameters for more than one substance from a single file is very straight forward: simply add more identifiers to the list.
Note that the **order** in which which identifiers are provided **is important**. When computing vector valued properties, **the order of the physical properties matches the order of the substances within the parameter object**.

In [9]:
# a system containing a ternary mixture
parameters = PcSaftParameters.from_json(
    ['methane', 'hexane', 'dodecane'], 
    pure_path=file_na
)
parameters

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|methane|16.043|1|3.7039|150.03|0|0|0|0|1|1|
|hexane|86.177|3.0576|3.7983|236.77|0|0|0|0|1|1|
|dodecane|170.338|5.306|3.8959|249.21|0|0|0|0|1|1|

### From multiple files

There may be cases where we have to split our parameter information across different files. For example, the `feos` repository has parameters stored in different files where each file corresponds to the parameter's original publication. Constructing the parameter object using multiple different json files is a bit more complicated. We can provide a list tuples, each of which contains the list of substances and the file where parameters are stored.

In the example below, we define a 4 component mixture from three input files:

- methane is read from a file containing non-associating substance parameters.
- parameters for 1-butanol and water are read from a file containing associating substances, and
- acetone parameters are read from a file that contains substances modelled with dipolar interactions.

In [11]:
# na = non-associating
# assoc = associating
file_na = '../parameters/pcsaft/gross2001.json'
file_assoc = '../parameters/pcsaft/gross2002.json'
file_dipolar = '../parameters/pcsaft/gross2006.json'

parameters = PcSaftParameters.from_multiple_json(
    [
        (['C'], file_na), 
        (['CCCCO', 'O'], file_assoc), 
        (['CC(C)=O'], file_dipolar)
    ], 
    search_option=IdentifierOption.Smiles
)
parameters

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|methane|16.043|1|3.7039|150.03|0|0|0|0|1|1|
|1-butanol|74.123|2.7515|3.6139|259.59|0|0|0.006692|2544.6|1|1|
|water|18.015|1.0656|3.0007|366.51|0|0|0.034868|2500.7|1|1|
|acetone|58.08|2.7447|3.2742|232.99|2.88|0|0|0|1|1|

### With binary interaction parameters

Some mixtures cannot be adequately described with combination rules from pure substance parameters.
In PC-SAFT, we can use a binary interaction parameter, `k_ij`, to enhance the description of mixture behavior.
These interaction parameters can be supplied from a json file via the `binary_path` option.

This parameter is not shown in the default representation of the parameter object. You can access the matrix of `k_ij` via the getter, `PcSaftParameters.k_ij`.

In [12]:
file_na = '../parameters/pcsaft/gross2001.json'
file_assoc = '../parameters/pcsaft/gross2002.json'
file_binary = '../parameters/pcsaft/gross2002_binary.json'

parameters = PcSaftParameters.from_multiple_json(
    [
        (['CCCC'], file_na), 
        (['CCCCO',], file_assoc)
    ],
    binary_path=file_binary,
    search_option=IdentifierOption.Smiles
)
parameters

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|butane|58.123|2.3316|3.7086|222.88|0|0|0|0|1|1|
|1-butanol|74.123|2.7515|3.6139|259.59|0|0|0.006692|2544.6|1|1|

In [13]:
parameters.k_ij

array([[0.   , 0.015],
       [0.015, 0.   ]])

---
## Building parameters in Python

Building `PcSaftParameters` in Python is a bit more involved since the `PcSaftParameters` object is built from multiple intermediate objects.
Let's import these objects, i.e.

- the `Identifier` object that stores information about how a substance can be identified,
- the `PcSaftRecord` object that stores our SAFT parameters,
- and the `PureRecord` object that bundles identifier and parameters together with the molar weight.

All these objects are imported from the `feos.pcsaft` module.

In [14]:
from feos.pcsaft import Identifier, PcSaftRecord, PureRecord

In [17]:
identifier = Identifier(
    cas='106-97-8',
    name='butane',
    iupac_name='butane',
    smiles='CCCC',
    inchi='InChI=1/C4H10/c1-3-4-2/h3-4H2,1-2H3',
    formula='C4H10'
)
identifier

Identifier(cas=106-97-8, name=butane, iupac_name=butane, smiles=CCCC, inchi=InChI=1/C4H10/c1-3-4-2/h3-4H2,1-2H3, formula=C4H10)

The `PcSaftRecord` contains the model parameters for a pure substance. *Mandatory* parameters are

- the number of segments, `m`, which is a dimensionless floating point number,
- the Lennard-Jones structure parameter (diameter), `sigma`, in units of Angstrom, and
- the Lennard-Jones energy parameter, `epsilon_k`, in units of Kelvin.

*Optional* parameters are

- the dipole moment, `mu`, in units of Debye used to model dipolar substances,
- the quadrupole moment, `q`, in units of Debye used to model quadrupolar substances,
- parameters to model association:
    - `kappa_ab`, `epsilon_k_ab`, `na`, `nb`
- and parameters for entropy scaling:
    - `viscosity`, `diffusion`, and `thermal_conductivity`
    - each of which is a list containing coefficients for the respective correlation functions.

In [18]:
# parameters for a non-associating, non-polar substance (butane)
psr = PcSaftRecord(m=2.3316, sigma=3.7086, epsilon_k=222.88)
psr

PcSaftRecord(m=2.3316, sigma=3.7086, epsilon_k=222.88)

A `PureRecord` is built from an `Identifier`, the molar weight (in gram per mole) and a `PcSaftRecord`. Optionally, but not shown in this example, we can provide an `ideal_gas_record` depending on the ideal gas model used in the equation of state. We will not discuss this contribution here but address the topic in a different example.

In [19]:
butane = PureRecord(identifier, molarweight=58.123, model_record=psr)
butane

PureRecord(
	identifier=Identifier(cas=106-97-8, name=butane, iupac_name=butane, smiles=CCCC, inchi=InChI=1/C4H10/c1-3-4-2/h3-4H2,1-2H3, formula=C4H10),
	molarweight=58.123,
	model_record=PcSaftRecord(m=2.3316, sigma=3.7086, epsilon_k=222.88),
)

#### `PcSaftParameters` for a single component 

For a single substance, we can use the `PcSaftParameters.new_pure` constructor.

In [20]:
parameters = PcSaftParameters.new_pure(butane)
parameters

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|butane|58.123|2.3316|3.7086|222.88|0|0|0|0|1|1|

#### `PcSaftParameters` for binary mixtures

We can create another `PureRecord` for a second component. Then, the `PcSaftParameters.new_binary` constructor let's us build the parameters. Optionally, we can also directly provide a `k_ij` value for this system. Here, we build an associating substance record using the `AssociationRecord` object.

In [26]:
butan_1_ol = PureRecord(
    identifier=Identifier(
        cas='71-36-3',
        name='1-butanol',
        iupac_name='butan-1-ol',
        smiles='CCCCO',
        inchi='InChI=1/C4H10O/c1-2-3-4-5/h5H,2-4H2,1H3',
        formula='C4H10O'
    ), 
    molarweight=74.123, 
    model_record=PcSaftRecord(
        m=2.7515, 
        sigma=3.6139, 
        epsilon_k=259.59, 
        association_record=AssociationRecord(kappa_ab=0.006692, epsilon_k_ab=2544.6)
    )
)

In [27]:
parameters = PcSaftParameters.new_binary([butane, butan_1_ol], binary_record=0.015)
parameters

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|butane|58.123|2.3316|3.7086|222.88|0|0|0|0|1|1|
|1-butanol|74.123|2.7515|3.6139|259.59|0|0|0.006692|2544.6|1|1|

In [28]:
parameters.k_ij

array([[0.   , 0.015],
       [0.015, 0.   ]])

#### `PcSaftParameters` for mixtures with more than two components

For mixtures with more than two components, we can use the `PcSaftParameters.from_records` constructor which takes a list of `PureRecords` and a `numpy.ndarray` containing the matrix of `k_ij` values.

In [30]:
import numpy as np

k_ij = np.zeros((2, 2))
k_ij[0, 1] = k_ij[1, 0] = 0.015

parameters = PcSaftParameters.from_records(
    [butane, butan_1_ol], 
    binary_records=k_ij
)
parameters

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|butane|58.123|2.3316|3.7086|222.88|0|0|0|0|1|1|
|1-butanol|74.123|2.7515|3.6139|259.59|0|0|0.006692|2544.6|1|1|

In [31]:
parameters.k_ij

array([[0.   , 0.015],
       [0.015, 0.   ]])

---
## Parameters from homo-segmented group contribution (homo-GC)

An alternative to substance specific parameters are parameters that combine information from functional groups (molecule *segments*).
A simple variant that only uses the *number of segments* (*not* how these segments are connected to form the molecule) is the so-called homo-segmented group contribution method (homo-GC).

As with regular SAFT parameters, we can build a `PcSaftParameters` object from json or from Python - using segment information.

### From json files

We need at least two files: 

- `pure_path`: a file containing the substance identifiers *and* the segments that form the molecule
- `segments_path`: a file that contains the segments (identifier and model parameters)

As before, we can specify our substance identifier using `search_option` and we can provide binary interaction parameters (segment-segment `k_ij`) via the `binary_path` argument.

In [32]:
pure_path = '../parameters/pcsaft/gc_substances.json'
segments_path = '../parameters/pcsaft/sauer2014_homo.json'

parameters = PcSaftParameters.from_json_segments(
    ['CCCC', 'CCCCO'], 
    pure_path, 
    segments_path, 
    search_option=IdentifierOption.Smiles
)
parameters

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|butane|58.122159999999994|2.1360799999999998|3.7945688260994523|233.79002902513017|0|0|0|0|1|1|
|1-butanol|74.12158|2.3821600000000003|3.7568140627964164|278.79916705846796|0|0|0.006825|2517|1|1|

### From Python

Building parameters in Python follows a similar approach as for regular parameters. To build `PcSaftParameters` from segments, we need to specify:

- The `ChemicalRecord` which contains the `Identifier` and the segments (as list of `str`s),
- and the `SegmentRecord` which specifies the identifier of the segment (has to be the same as in the list of the `ChemicalRecord`), the molar weight and the `PcSaftRecord` for the segment.

If both are available, we can use the `PcSaftParameters.from_segments` constructor to build the parameters.

In [33]:
from feos.pcsaft import ChemicalRecord, SegmentRecord

cr1 = ChemicalRecord(
    identifier=Identifier(
    cas='106-97-8',
    name='butane',
    iupac_name='butane',
    smiles='CCCC',
    inchi='InChI=1/C4H10/c1-3-4-2/h3-4H2,1-2H3',
    formula='C4H10'
),
    segments=['CH3', 'CH2', 'CH2', 'CH3']
)

cr2 = ChemicalRecord(
    identifier=Identifier(
        cas='71-36-3',
        name='1-butanol',
        iupac_name='butan-1-ol',
        smiles='CCCCO',
        inchi='InChI=1/C4H10O/c1-2-3-4-5/h5H,2-4H2,1H3',
        formula='C4H10O'
    ),
    segments=['CH3', 'CH2', 'CH2', 'CH2', 'OH']
)

Each segment has a `PcSaftRecord` which can be constructed just like we did before for a substance.

In [35]:
ch3 = SegmentRecord(
    'CH3', 
    molarweight=15.0345, 
    model_record=PcSaftRecord(m=0.61198, sigma=3.7202, epsilon_k=229.90)
)
ch2 = SegmentRecord(
    'CH2', 
    molarweight=14.02658, 
    model_record=PcSaftRecord(m=0.45606, sigma=3.8900, epsilon_k=239.01)
)
oh = SegmentRecord(
    'OH', 
    molarweight=17.00734, 
    model_record=PcSaftRecord(
        m=0.40200, 
        sigma=3.2859, 
        epsilon_k=488.66,
        association_record=AssociationRecord(
            epsilon_k_ab=2517.0, 
            kappa_ab=0.006825
        )
    )
)

In [36]:
parameters = PcSaftParameters.from_segments(
    chemical_records=[cr1, cr2], 
    segment_records=[ch3, ch2, oh]
)
parameters

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|butane|58.122159999999994|2.1360799999999998|3.7945688260994523|233.79002902513017|0|0|0|0|1|1|
|1-butanol|74.12158|2.3821600000000003|3.7568140627964164|278.79916705846796|0|0|0.006825|2517|1|1|

## Accessing information from parameter objects

Once the `PcSaftParameter` object is constructed, within a jupyter notebook, we get a nice representation in form of a markdown table.
Sometimes, however you might want to access information not presented in this table or you might want to store information in a variable.

Let's build parameters for the four-component mixture we looked at earlier:

In [38]:
file_na = '../parameters/pcsaft/gross2001.json'
file_assoc = '../parameters/pcsaft/gross2002.json'
file_dipolar = '../parameters/pcsaft/gross2006.json'

parameters = PcSaftParameters.from_multiple_json(
    [
        (['C'], file_na), 
        (['CCCCO', 'O'], file_assoc), 
        (['CC(C)=O'], file_dipolar)
    ], 
    search_option=IdentifierOption.Smiles
)
parameters

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|
|-|-|-|-|-|-|-|-|-|-|-|
|methane|16.043|1|3.7039|150.03|0|0|0|0|1|1|
|1-butanol|74.123|2.7515|3.6139|259.59|0|0|0.006692|2544.6|1|1|
|water|18.015|1.0656|3.0007|366.51|0|0|0.034868|2500.7|1|1|
|acetone|58.08|2.7447|3.2742|232.99|2.88|0|0|0|1|1|

As we've seen before, we can directly access the binary interaction parameter, `k_ij`, which is zero here for all binary interactions (we did not provide a file).

In [39]:
parameters.k_ij

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

### Get `PureRecord`s via `parameters.pure_records`

We have seen above that it is possible to generate parameters in Python using intermediate objects, such as `Identifier`, `PcSaftRecord` and `PureRecord`.
You can generate these objects for all substances via the `pure_records` method (getter).
This getter returns `PureRecord` objects which can be further deconstructed to yield the `Identifier` and  `PcSaftRecord` objects and the molar weight.

Note that the order in which substances are returned matches the order in which we specified the substances above.

In [40]:
parameters.pure_records

[PureRecord(
 	identifier=Identifier(cas=74-82-8, name=methane, iupac_name=methane, smiles=C, inchi=InChI=1/CH4/h1H4, formula=CH4),
 	molarweight=16.043,
 	model_record=PcSaftRecord(m=1, sigma=3.7039, epsilon_k=150.03),
 ),
 PureRecord(
 	identifier=Identifier(cas=71-36-3, name=1-butanol, iupac_name=butan-1-ol, smiles=CCCCO, inchi=InChI=1/C4H10O/c1-2-3-4-5/h5H,2-4H2,1H3, formula=C4H10O),
 	molarweight=74.123,
 	model_record=PcSaftRecord(m=2.7515, sigma=3.6139, epsilon_k=259.59, association_record=AssociationRecord(kappa_ab=0.006692, epsilon_k_ab=2544.6, na=1, nb=1)),
 ),
 PureRecord(
 	identifier=Identifier(cas=7732-18-5, name=water, iupac_name=oxidane, smiles=O, inchi=InChI=1/H2O/h1H2, formula=H2O),
 	molarweight=18.015,
 	model_record=PcSaftRecord(m=1.0656, sigma=3.0007, epsilon_k=366.51, association_record=AssociationRecord(kappa_ab=0.034868, epsilon_k_ab=2500.7, na=1, nb=1)),
 ),
 PureRecord(
 	identifier=Identifier(cas=67-64-1, name=acetone, iupac_name=propan-2-one, smiles=CC(C)=O

In [41]:
for pure_record in parameters.pure_records:
    print(f"σ ({pure_record.identifier.name})\t = {pure_record.model_record.sigma} A")

σ (methane)	 = 3.7039 A
σ (1-butanol)	 = 3.6139 A
σ (water)	 = 3.0007 A
σ (acetone)	 = 3.2742 A


In [42]:
# get identifier of substance 0
parameters.pure_records[0].identifier

Identifier(cas=74-82-8, name=methane, iupac_name=methane, smiles=C, inchi=InChI=1/CH4/h1H4, formula=CH4)

In [43]:
# get molarweight of substance 0
parameters.pure_records[0].molarweight

16.043

A `PureRecord` object can be used to generate a json string which then can conveniently be stored in a file.

In [44]:
# generate a json string from identifier of substance 0
parameters.pure_records[0].to_json_str()

'{"identifier":{"cas":"74-82-8","name":"methane","iupac_name":"methane","smiles":"C","inchi":"InChI=1/CH4/h1H4","formula":"CH4"},"molarweight":16.043,"model_record":{"m":1.0,"sigma":3.7039,"epsilon_k":150.03}}'

## Summary

In $\text{FeO}_\text{s}$, handling of parameters is not predefined within the underlying library. Each implementation of an equation of state has complete autonomy about the way parameters are handled.
Howevery, we offer some convenient ways to automatize parameter handling which result in the methods presented in this example.

We looked at different ways to get parameters for the `EquationOfState.pcsaft()` equation of state, i.e.

- how to read parameters for substances from one or more files where we use the json format to store information, and
- how to generate parameters the same parameters in Python.
- In a similar fashion, we showed how parameters can be assembled using a homo-GC method.

Once parameters are created, we can retrieve information by extracting the intermediate objects such as `PureRecord`, `Identifier` and `PcSaftRecord`.

## Further reading

- For more information about group contribution methods, see [this paper by Sauer et al.](https://pubs.acs.org/doi/abs/10.1021/ie502203w)
- Files of published parameters can be found [in the `feos` github repositoy](https://github.com/feos-org/feos/tree/main/parameters/pcsaft).

## Concluding remkars

Hopefully you found this example helpful. If you have comments, critique or feedback, please let us know and consider [opening an issue on github](https://github.com/feos-org/feos/issues).